In [1]:
import pandas as pd
df=pd.read_fwf('text.txt',sep=',',encoding='latin-1')
df.to_csv('log.csv')
df.head()

,"text,label",Unnamed: 1,Unnamed: 2
0,Stuning even for the non-gamer: This sound tra...,NaN,NaN
1,""" The best soundtrack ever to anything.: I'm r...",NaN,NaN
2,""" Amazing!: This soundtrack is my favorite mus...",NaN,NaN
3,""" Excellent Soundtrack: I truly like this soun...",NaN,NaN
4,""" Remember, Pull Your Jaw Off The Floor After ...",NaN,NaN


In [2]:
df.drop('Unnamed: 2',axis=1,inplace=True)
df.head(5)

,"text,label",Unnamed: 1
0,Stuning even for the non-gamer: This sound tra...,NaN
1,""" The best soundtrack ever to anything.: I'm r...",NaN
2,""" Amazing!: This soundtrack is my favorite mus...",NaN
3,""" Excellent Soundtrack: I truly like this soun...",NaN
4,""" Remember, Pull Your Jaw Off The Floor After ...",NaN


In [3]:
x=[]
for entry in df['text,label']:
    x.append(entry.split('__')[-1])

In [4]:
df['Unnamed: 1']=x

In [5]:
df.head(10)

,"text,label",Unnamed: 1
0,Stuning even for the non-gamer: This sound tra...,2
1,""" The best soundtrack ever to anything.: I'm r...",2
2,""" Amazing!: This soundtrack is my favorite mus...",2
3,""" Excellent Soundtrack: I truly like this soun...",2
4,""" Remember, Pull Your Jaw Off The Floor After ...",2
5,""" an absolute masterpiece: I am quite sure any...",2
6,""" Buyer beware: This is a self-published book,...",1
7,""" Glorious story: I loved Whisper of the wicke...",2
8,""" A FIVE STAR BOOK: I just finished reading Wh...",2
9,""" Whispers of the Wicked Saints: This was a ea...",2


In [6]:
df.columns=['text','label']
df.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,2
1,""" The best soundtrack ever to anything.: I'm r...",2
2,""" Amazing!: This soundtrack is my favorite mus...",2
3,""" Excellent Soundtrack: I truly like this soun...",2
4,""" Remember, Pull Your Jaw Off The Floor After ...",2


In [7]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [8]:
np.random.seed(500)

In [9]:
#removing blank rows if any
df['text'].dropna(inplace=True)

In [10]:
#changing text to lowercase
df['text']=[entry.lower() for entry in df['text']]

In [11]:
#breaking a stream of text into words,phrases,symbols etc
df['text']=[word_tokenize(entry) for entry in df['text']]

In [12]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    
    df.loc[index,'text_final'] = str(Final_words)

In [ ]:
###isalpha() methods returns “True” if all characters in the string are alphabets.
###for stopwords.words('english') it contains:
#{‘ourselves’, ‘hers’, ‘between’, ‘yourself’, ‘but’, ‘again’, ‘there’, ‘about’, ‘once’, ‘during’, ‘out’, ‘very’, ‘having’,
#‘with’, ‘they’, ‘own’, ‘an’, ‘be’, ‘some’, ‘for’, ‘do’, ‘its’, ‘yours’, ‘such’, ‘into’, ‘of’, ‘most’, ‘itself’, ‘other’, 
#‘off’, ‘is’, ‘s’, ‘am’, ‘or’, ‘who’, ‘as’, ‘from’, ‘him’, ‘each’, ‘the’, ‘themselves’, ‘until’, ‘below’, ‘are’, ‘we’,
#‘these’, ‘your’, ‘his’, ‘through’, ‘don’, ‘nor’, ‘me’, ‘were’, ‘her’, ‘more’, ‘himself’, ‘this’, ‘down’, ‘should’, ‘our’,
#‘their’, ‘while’, ‘above’, ‘both’, ‘up’, ‘to’, ‘ours’, ‘had’, ‘she’, ‘all’, ‘no’, ‘when’, ‘at’, ‘any’, ‘before’, ‘them’,
#‘same’, ‘and’, ‘been’, ‘have’, ‘in’, ‘will’, ‘on’, ‘does’, ‘yourselves’, ‘then’, ‘that’, ‘because’, ‘what’, ‘over’, ‘why’,
#‘so’, ‘can’, ‘did’, ‘not’, ‘now’, ‘under’, ‘he’, ‘you’, ‘herself’, ‘has’, ‘just’, ‘where’, ‘too’, ‘only’, ‘myself’, ‘which’,
#‘those’, ‘i’, ‘after’, ‘few’, ‘whom’, ‘t’, ‘being’, ‘if’, ‘theirs’, ‘my’, ‘against’, ‘a’, ‘by’, ‘doing’, ‘it’, ‘how’,
#‘further’, ‘was’, ‘here’, ‘than’}

In [13]:
df.head()

,text,label,text_final
0,"[stuning, even, for, the, non-gamer, :, this, ...",2,"['stun', 'even', 'sound', 'track', 'beautiful'..."
1,"[``, the, best, soundtrack, ever, to, anything...",2,"['best', 'soundtrack', 'ever', 'anything', 're..."
2,"[``, amazing, !, :, this, soundtrack, is, my, ...",2,"['amazing', 'soundtrack', 'favorite', 'music',..."
3,"[``, excellent, soundtrack, :, i, truly, like,...",2,"['excellent', 'soundtrack', 'truly', 'like', '..."
4,"[``, remember, ,, pull, your, jaw, off, the, f...",2,"['remember', 'pull', 'jaw', 'floor', 'hear', '..."


In [14]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text_final'],df['label'],test_size=0.3)

In [15]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [16]:
print(Tfidf_vect.vocabulary_)

{'even': 1533, 'sound': 4131, 'track': 4566, 'beautiful': 379, 'paint': 3170, 'mind': 2837, 'well': 4867, 'would': 4952, 'recomend': 3606, 'people': 3237, 'hate': 2064, 'video': 4775, 'game': 1867, 'music': 2928, 'play': 3305, 'cross': 1024, 'ever': 1537, 'best': 421, 'back': 321, 'away': 311, 'crude': 1027, 'take': 4376, 'fresh': 1827, 'step': 4220, 'guitar': 2015, 'soulful': 4128, 'orchestra': 3104, 'impress': 2240, 'anyone': 195, 'care': 638, 'listen': 2600, 'soundtrack': 4133, 'anything': 196, 'read': 3572, 'lot': 2645, 'review': 3722, 'say': 3839, 'figure': 1699, 'write': 4958, 'disagree': 1230, 'bit': 442, 'ultimate': 4647, 'masterpiece': 2746, 'timeless': 4510, 'year': 4977, 'beauty': 381, 'simply': 4013, 'refuse': 3631, 'price': 3410, 'tag': 4375, 'pretty': 3405, 'must': 2934, 'go': 1938, 'buy': 593, 'cd': 671, 'much': 2918, 'money': 2875, 'one': 3082, 'feel': 1676, 'worth': 4948, 'every': 1539, 'penny': 3236, 'amazing': 148, 'favorite': 1665, 'time': 4509, 'hand': 2035, 'inten

In [17]:
print(Train_X_Tfidf)

  (0, 4510)	0.37585200906943095
  (0, 4509)	0.15067952202110121
  (0, 3977)	0.35824282579770717
  (0, 3891)	0.2513701337444424
  (0, 3859)	0.2687337842782034
  (0, 3745)	0.3474159305568934
  (0, 3662)	0.289667706026126
  (0, 3572)	0.2948447699589719
  (0, 2934)	0.22985922881362789
  (0, 1946)	0.13426597012551528
  (0, 1533)	0.1775340408664745
  (0, 517)	0.3212236390393022
  (0, 487)	0.12311702495396104
  (0, 232)	0.24500246746075766
  (1, 4703)	0.3690316138310882
  (1, 4071)	0.6155404378897409
  (1, 3437)	0.36938346726278787
  (1, 2577)	0.3753782739477479
  (1, 1252)	0.36005273457651366
  (1, 593)	0.27916781906343274
  (2, 4749)	0.183519759894041
  (2, 4639)	0.1499621052111866
  (2, 4471)	0.10288372455294761
  (2, 4205)	0.11745195236813158
  (2, 3851)	0.23019983004363131
  :	:
  (6998, 2504)	0.11506270305235244
  (6998, 2128)	0.136414816457353
  (6998, 1982)	0.07127177033239217
  (6998, 1787)	0.2197802296099393
  (6998, 1752)	0.19903031224918907
  (6998, 1715)	0.13512414483053384
  (69

In [18]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(Test_Y,predictions_NB)*100)

Naive Bayes Accuracy Score ->  80.06666666666666
